In [41]:
from enum import Enum, member

class Modo(Enum):
    """
    Formas de uso de un reloj inteligente.
    """

    NORMAL: member = 1
    EJERCICIO: member = 2
    REPOSO: member = 3

In [42]:
# PREINICIALIZACIÓN SIN SENTIDO, SÓLO PORQUE ES UN NOTEBOOK
class Reloj:
    def __init__(self, material: str):
        self.material = material
        self.color_fondo = "blanco"
        self.medidas = {}

In [43]:
from abc import ABC, abstractmethod 

class RelojException(Exception):
    """
    Informa de acciones no permitidas en la comunicación
    entre el reloj y el celular.
    """
    pass

class ModoReloj(ABC):
    """
    Clase base abstracta para los modos del reloj.
    """
    def __init__(self, reloj: Reloj):
        self.reloj= reloj

    @abstractmethod
    def mostrar_mensaje(self, mensaje: str, tipo_mensaje: str): ...

    @abstractmethod
    def obtener_medida(self, nombre_medida: str) -> float: ...

In [44]:
class ModoNormal(ModoReloj):
    def mostrar_mensaje(self, mensaje: str, tipo_mensaje: str):
        print(mensaje)

    def obtener_medida(self, nombre_medida: str) -> float:
        return self.reloj.medidas.get(nombre_medida)


class ModoEjercicio(ModoReloj):
    def mostrar_mensaje(self, mensaje: str, tipo_mensaje: str):
        if tipo_mensaje == "llamada":
            print(mensaje)

    def obtener_medida(self, nombre_medida: str) -> float:
        return self.reloj.medidas.get(nombre_medida)


class ModoReposo(ModoReloj):
    def mostrar_mensaje(self, mensaje: str, tipo_mensaje: str):
        if tipo_mensaje == "alarma":
            print(mensaje)

    def obtener_medida(self, nombre_medida: str) -> float:
        raise RelojException("No se pueden obtener medidas en modo reposo")

In [45]:
class Reloj:
    def __init__(self, material: str):
        self.material = material
        self.estado_actual = ModoNormal(self)  # Estado inicial
        self.color_fondo = "blanco"
        self.medidas = {}

    def mostrar_mensaje(self, mensaje: str, tipo_mensaje: str):
        self.estado_actual.mostrar_mensaje(mensaje, tipo_mensaje)

    def cambiar_modo(self, nuevo_modo: str):
        modos_operacion = {
            Modo.REPOSO: ModoReposo(self),
            Modo.NORMAL: ModoNormal(self),
            Modo.EJERCICIO: ModoEjercicio(self),
        }
        if nuevo_modo not in modos_operacion.keys():
            raise RelojException
        return modos_operacion[nuevo_modo] 
            

    def obtener_medida(self, nombre_medida: str) -> float:
        return self.estado_actual.obtener_medida(nombre_medida)

    def adicionar_medida(self, nombre_medida: str, valor: float):
        self.medidas[nombre_medida] = valor

In [46]:
class Celular:
    """
    Celular inteligente que puede comunicarse con un reloj
    para obtener o mostrar información.
    """
    def __init__(self, marca: str):
        self.marca = marca
        self.reloj_asociado = None

    def asociar_reloj(self, reloj: Reloj):
        self.reloj_asociado = reloj

    def mostrar_medidas(self):
        """
        Muestra un conjunto básico de medidas del reloj.
        """
        if self.reloj_asociado is not None:
            try:
                print("Pasos: " + str(self.reloj_asociado.obtener_medida("pasos")))
                print("Presión: " + str(self.reloj_asociado.obtener_medida("presión")))
            except RelojException as error_medida:
                print(error_medida)
        else:
            print("No hay reloj asociado.")

    def llamada_entrante(self):
        if self.reloj_asociado is not None:
            print("Se envía llamada al reloj")
            self.reloj_asociado.mostrar_mensaje("Llamada entrante", "llamada")

    def alarma(self):
        if self.reloj_asociado is not None:
            print("Se envía alarma al reloj")
            self.reloj_asociado.mostrar_mensaje("Una alarma", "alarma")

In [47]:
import unittest


class TestReloj(unittest.TestCase):
    def test_obtener_medidas_modo_normal(self):
        reloj = Reloj("Madera")
        valor_pasos = reloj.obtener_medida("pasos")
        self.assertIsNone(valor_pasos)
        reloj.adicionar_medida("pasos", 100)
        valor_pasos = reloj.obtener_medida("pasos")
        self.assertEqual(100, valor_pasos)

    def test_obtener_medidas_modo_ejercicio(self):
        reloj = Reloj("Madera")
        reloj.cambiar_modo(Modo.EJERCICIO)
        valor_pasos = reloj.obtener_medida("pasos")
        self.assertIsNone(valor_pasos)
        reloj.adicionar_medida("pasos", 100)
        valor_pasos = reloj.obtener_medida("pasos")
        self.assertEqual(100, valor_pasos)


    def test_obtener_medidas_modo_reposo(self):
        reloj = Reloj("Madera")
        reloj.cambiar_modo(Modo.REPOSO)
        with self.assertRaises(RelojException):
            reloj.obtener_medida("pasos")

    def test_color_fondo_normal(self):
        reloj = Reloj("Madera")
        self.assertEqual("blanco", reloj.color_fondo)
        
# Conservar el ENUM
pruebas = TestReloj()
pruebas.test_obtener_medidas_modo_reposo()
pruebas.test_obtener_medidas_modo_normal()
pruebas.test_obtener_medidas_modo_ejercicio()



Modo.REPOSO Modo.NORMAL Modo.EJERCICIO
3 1 2


AssertionError: RelojException not raised

In [33]:
celular = Celular("gato")
reloj = Reloj("Madera")
reloj.adicionar_medida("pasos", 500)
reloj.adicionar_medida("presión", 70)
celular.reloj_asociado = reloj

print("---EN MODO NORMAL---")
celular.llamada_entrante()
celular.alarma()
celular.mostrar_medidas()
print()

print("---EN MODO EJERCICIO---")
reloj.cambiar_modo(Modo.EJERCICIO)
celular.llamada_entrante()
celular.alarma()
celular.mostrar_medidas()
print()

print("---EN MODO REPOSO---")
reloj.cambiar_modo(Modo.REPOSO)
celular.llamada_entrante()
celular.alarma()
celular.mostrar_medidas()
print()


---EN MODO NORMAL---
Se envía llamada al reloj
Llamada entrante
Se envía alarma al reloj
Una alarma
Pasos: 500
Presión: 70

---EN MODO EJERCICIO---
Se envía llamada al reloj
Llamada entrante
Se envía alarma al reloj
Pasos: 500
Presión: 70

---EN MODO REPOSO---
Se envía llamada al reloj
Se envía alarma al reloj
Una alarma
No se pueden obtener medidas en modo reposo

